In [21]:
import os
from openai import OpenAI
import pandas as pd
import re
from dotenv import load_dotenv
import json

# 1) Load environment variables
load_dotenv()
OPENAI_MODEL = 'gpt-4o'  # as per your example

client = OpenAI(api_key=os.getenv('OPENAI_APIKEY'))


def call_openai_api(system_prompt, user_prompt):
    """
    Calls the OpenAI ChatCompletion endpoint and returns the full response object.
    This is a separate function so we can debug the raw response before JSON parsing.
    """
    try:
        response = client.chat.completions.create(
            model=OPENAI_MODEL,
            temperature=0.1,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
           # response_format={"type": "json_object"}  # optional if your endpoint supports this
        )
        return response
    except Exception as e:
        print(f"Error calling OpenAI API: {e}")
        return None


def extract_requirements_from_row(paper_name, row_text):
    """
    Sends a row’s text (with requirements) to the OpenAI ChatCompletion endpoint
    to extract structured data in JSON. 
    Returns a Python list of dictionaries with the columns:
      - Requirement
      - Found_Content_With_Page
      - Paper
    """
    system_prompt = (
        "You are a data-extraction assistant. You receive text describing requirements found in a paper. "
        "Your goal is to extract structured data in strict JSON format."
    )

    user_prompt = f"""
Extract **all** requirements from the text. 
- Each requirement is marked with the pattern (R<Number>), for example (R4), (R12), etc.
- The text may contain multiple requirements; you must find **all** occurrences.
- For each requirement found, return an object with the following keys:
  1) "Requirement": the requirement code (e.g. "R4"),
  2) "Found_Content_With_Page": the text associated with that requirement (including page number),
  3) "Paper": the provided paper filename.

Each object should correspond to **one** requirement from the text.
Return these objects in a JSON array. That means if you find 5 requirements, return an array of length 5. Output only valid JSON, with no extra text or code fences.

**Important**: Do not stop after finding the first requirement. Keep scanning the text until you have processed every instance of the pattern (R\d+). If no requirements are found, return an empty array `[]`.

Here is an example with **multiple** requirements:

**Example of desired JSON output**:
[
  {{
    "Requirement": "R1",
    "Found_Content_With_Page": "Discusses productivity improvements (p. 1).",
    "Paper": "MyPaper.pdf"
  }},
  {{
    "Requirement": "R4",
    "Found_Content_With_Page": "Mentions cost considerations (p. 2).",
    "Paper": "MyPaper.pdf"
  }},
  {{
    "Requirement": "R10",
    "Found_Content_With_Page": "Highlights organizational acceptance (p. 6).",
    "Paper": "MyPaper.pdf"
  }}
]

PAPER NAME: {paper_name}
TEXT:
{row_text}
"""
    print ("User Prompt: ", user_prompt)
    # 1) Call the API
    response = call_openai_api(system_prompt, user_prompt)
    if response is None:
        print("No response from API, returning empty list.")
        return []

    # 2) Inspect the raw response for debugging
    print("--------------- RAW RESPONSE ---------------")
    print(response)
    print("--------------------------------------------")
    # Typically, the content is in `response.choices[0].message.content`
    json_string = response.choices[0].message.content
    # Remove code fences if present
    json_string_clean = re.sub(r"^```[a-zA-Z]*\n?", "", json_string.strip())
    json_string_clean = re.sub(r"```$", "", json_string_clean).strip()

    # 3) Attempt to extract the JSON from the response
    try:
        print("----- RAW CONTENT FROM THE MODEL -----")
        print(json_string_clean)
        print("--------------------------------------")

        # 4) Parse the JSON string
        extracted_data = json.loads(json_string_clean)

        # 5) If it's a dict, wrap in a list to normalize
        if isinstance(extracted_data, dict):
            extracted_data = [extracted_data]

        # 6) Confirm success
        print(f"Extracted {len(extracted_data)} requirements from '{paper_name}'")
        return extracted_data

    except json.JSONDecodeError as e:
        print("Error loading JSON: ", e)
        print("Raw response content was:\n", response.choices[0].message.content)
        return []
    except Exception as e:
        print(f"Unexpected error parsing JSON: {e}")
        return []


def process_excel(input_excel="req_matches.xlsx", output_excel="req_matches_parsed.xlsx", n_rows=3):
    """
    Main function to:
    1) Read an input Excel file
    2) Extract requirement data from each row
    3) Write results to a new Excel file
    """
    # 2) Read the original Excel file
    df_input = pd.read_excel(input_excel)

    # Suppose your DataFrame has columns ["Paper", "Requirements Match"]
    # Adjust as needed based on your actual column names
    all_extracted_rows = []

    # If you'd like to process all rows, remove .head(n_rows)
    for idx, row in df_input.head(n_rows).iterrows():
        paper_name = row["Paper"]  # adjust column name if different
        row_text = row["Requirements Match"]  # adjust column name if different

        # 3) Extract the requirements via OpenAI
        extracted_list = extract_requirements_from_row(paper_name, row_text)

        # 4) Append each extracted requirement to a master list
        for item in extracted_list:
            all_extracted_rows.append(item)

    # 5) Convert to DataFrame & save
    df_output = pd.DataFrame(
        all_extracted_rows,
        columns=["Requirement", "Found_Content_With_Page", "Paper"]
    )

    # 6) Write the result to a new Excel
    df_output.to_excel(output_excel, index=False)
    print(f"Results written to {output_excel}")


In [22]:
process_excel(
    input_excel="req_matches.xlsx",
    output_excel="req_matches_parsed_2.xlsx",
    n_rows=50  # or however many rows you want to process
)


User Prompt:  
Extract **all** requirements from the text. 
- Each requirement is marked with the pattern (R<Number>), for example (R4), (R12), etc.
- The text may contain multiple requirements; you must find **all** occurrences.
- For each requirement found, return an object with the following keys:
  1) "Requirement": the requirement code (e.g. "R4"),
  2) "Found_Content_With_Page": the text associated with that requirement (including page number),
  3) "Paper": the provided paper filename.

Each object should correspond to **one** requirement from the text.
Return these objects in a JSON array. That means if you find 5 requirements, return an array of length 5. Output only valid JSON, with no extra text or code fences.

**Important**: Do not stop after finding the first requirement. Keep scanning the text until you have processed every instance of the pattern (R\d+). If no requirements are found, return an empty array `[]`.

Here is an example with **multiple** requirements:

**Exam